<a href="https://colab.research.google.com/github/scj02/GemeloDigital/blob/Procesos_Auditoria/Prueba_Gemelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install web3

In [20]:
import hashlib
import json
import time
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.exceptions import InvalidSignature

# Web3 (requiere conexión a un proveedor como Infura o Alchemy en testnet)
from web3 import Web3

In [27]:
# Gemelo Digital inicial
diploma = {
    "id": "DIP-002",
    "estudiante": "Johan Ayala",
    "programa": "Ingeniería de Sistemas",
    "estado": "En revisión",
    "atributos": {
        "paz_y_salvo": True,
        "notas_completas": True,
        "deudas": False,
        "creditos_cumplidos": 160,
        "creditos_totales": 160,
        "certificado_idioma": False,   # Nuevo atributo
        "saber_pro": True              # Nuevo atributo
    },
    "historial": []
}

print("Gemelo Digital inicial:", json.dumps(diploma, indent=2, ensure_ascii=False))

Gemelo Digital inicial: {
  "id": "DIP-002",
  "estudiante": "Johan Ayala",
  "programa": "Ingeniería de Sistemas",
  "estado": "En revisión",
  "atributos": {
    "paz_y_salvo": true,
    "notas_completas": true,
    "deudas": false,
    "creditos_cumplidos": 160,
    "creditos_totales": 160,
    "certificado_idioma": false,
    "saber_pro": true
  },
  "historial": []
}


In [28]:
def actualizar_estado(diploma, nuevo_estado):
    diploma["estado"] = nuevo_estado
    diploma["historial"].append({
        "estado": nuevo_estado,
        "timestamp": datetime.utcnow().isoformat() + "Z"
    })

In [29]:
def validar_diploma(d):
    reglas = []

    # --- Reglas de validación básicas ---
    if not d["atributos"]["paz_y_salvo"]:
        reglas.append("🚩 No está paz y salvo (pendiente de biblioteca o pagos)")

    if d["atributos"]["deudas"]:
        reglas.append("🚩 Tiene deudas pendientes con la universidad")

    if not d["atributos"]["notas_completas"]:
        reglas.append("🚩 No completó todas las asignaturas del plan de estudios")

    if d["atributos"]["creditos_cumplidos"] < d["atributos"]["creditos_totales"]:
        reglas.append("🚩 No ha completado todos los créditos requeridos")

    if d["atributos"]["creditos_cumplidos"] > d["atributos"]["creditos_totales"]:
        reglas.append("🚩 Créditos exceden lo requerido (posible error o fraude)")

    if d["atributos"]["creditos_totales"] != 160:
        reglas.append("🚩 El total de créditos no coincide con el plan oficial (160)")

    # --- Nuevos criterios ---
    if not d["atributos"].get("certificado_idioma", False):
        reglas.append("🚩 No cumple requisito de certificado de idioma")

    if not d["atributos"].get("saber_pro", False):
        reglas.append("🚩 No aprobó el examen Saber Pro")

    return reglas


# --- Ejecución de validación ---
banderas = validar_diploma(diploma)

if banderas:
    actualizar_estado(diploma, "Bloqueado - Banderas rojas")
    print("❌ Integridad comprometida. Motivos detectados:")
    for b in banderas:
        print("   ", b)
else:
    actualizar_estado(diploma, "Apto para emisión")
    print("✅ Diploma válido e íntegro")

❌ Integridad comprometida. Motivos detectados:
    🚩 No cumple requisito de certificado de idioma


/tmp/ipython-input-4042965120.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z"


In [30]:
# Serialización determinista y hash
diploma_json = json.dumps(diploma, sort_keys=True, ensure_ascii=False).encode()
hash_diploma = hashlib.sha256(diploma_json).hexdigest()

diploma["hash"] = hash_diploma
diploma["historial"].append({
    "evento": "Hash generado",
    "hash": hash_diploma,
    "timestamp": datetime.utcnow().isoformat() + "Z"
})

print("Gemelo Digital actualizado:", json.dumps(diploma, indent=2, ensure_ascii=False))
print("Hash anclado (simulado blockchain):", hash_diploma)

Gemelo Digital actualizado: {
  "id": "DIP-002",
  "estudiante": "Johan Ayala",
  "programa": "Ingeniería de Sistemas",
  "estado": "Bloqueado - Banderas rojas",
  "atributos": {
    "paz_y_salvo": true,
    "notas_completas": true,
    "deudas": false,
    "creditos_cumplidos": 160,
    "creditos_totales": 160,
    "certificado_idioma": false,
    "saber_pro": true
  },
  "historial": [
    {
      "estado": "Bloqueado - Banderas rojas",
      "timestamp": "2025-10-17T08:39:57.308994Z"
    },
    {
      "evento": "Hash generado",
      "hash": "3687212801ff8c0dd0fc616008f452e1bf1a8a5dc034176012be3712fbc86bcb",
      "timestamp": "2025-10-17T08:40:13.277885Z"
    }
  ],
  "hash": "3687212801ff8c0dd0fc616008f452e1bf1a8a5dc034176012be3712fbc86bcb"
}
Hash anclado (simulado blockchain): 3687212801ff8c0dd0fc616008f452e1bf1a8a5dc034176012be3712fbc86bcb


/tmp/ipython-input-2169702725.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z"


In [31]:
# Generación de claves ECDSA (p256)
private_key = ec.generate_private_key(ec.SECP256R1())
public_key = private_key.public_key()

# Firmar hash
hash_bytes = bytes.fromhex(hash_diploma)
signature = private_key.sign(hash_bytes, ec.ECDSA(hashes.SHA256()))

# Verificar firma
try:
    public_key.verify(signature, hash_bytes, ec.ECDSA(hashes.SHA256()))
    print("✅ Firma verificada correctamente")
except InvalidSignature:
    print("❌ Firma inválida")

# Exportar clave pública PEM
pem_pub = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)
print(pem_pub.decode())

✅ Firma verificada correctamente
-----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAER4e/T1cQXJK+HI0vNIbs0yPWsW1H
axzOpuROFTw3QCQi1ip0bM5X+uCK/BgCgmoyLCTPNz5bY9w9No/iq99w0Q==
-----END PUBLIC KEY-----



In [32]:
# Conectar a Sepolia con Infura
infura_url = "https://sepolia.infura.io/v3/1e0a83b4b7ea4916ba64da9e4d1213a9"
w3 = Web3(Web3.HTTPProvider(infura_url))

print("¿Conectado?", w3.is_connected())

# Cuenta (usa tu clave privada en formato HEX, no en Base64)
private_key = "0x4c0883a69102937d6231471b5dbb6204fe5129617082797a64894ccf37d85a7a"  # EJEMPLO
acct = w3.eth.account.from_key(private_key)
print("Dirección:", acct.address)

# Hash del diploma (simulado con SHA-256 de un texto)
import hashlib
hash_diploma = hashlib.sha256(b"Diploma Johan Ayala 2025").hexdigest()
print("Hash del diploma:", hash_diploma)

# Construir la transacción
tx = {
    'to': acct.address,   # puedes enviarla a ti mismo
    'value': 0,           # sin ETH, solo datos
    'data': bytes.fromhex(hash_diploma),  # guardar el hash en data
    'gas': 50000,
    'gasPrice': w3.to_wei('20', 'gwei'),  # Increased gas price
    'nonce': w3.eth.get_transaction_count(acct.address)
}

# Firmar y enviar
signed_tx = w3.eth.account.sign_transaction(tx, private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

print("Transacción enviada. Hash:", tx_hash.hex())

¿Conectado? True
Dirección: 0x1C98cfeaEc61518F3f80a1aD9f93E1E42AAE62CB
Hash del diploma: 3a764b52a5debaa4ab6114a66f5726a38b9f69e432f3770c53350a73bf48b838


Web3RPCError: {'code': -32000, 'message': 'insufficient funds for gas * price + value: balance 0, tx cost 1000000000000000, overshot 1000000000000000'}

In [33]:
df = pd.DataFrame([{
    "id": diploma["id"],
    "estudiante": diploma["estudiante"],
    "programa": diploma["programa"],
    "estado": diploma["estado"],
    "hash": diploma["hash"],
    "historial": json.dumps(diploma["historial"], ensure_ascii=False)
}])
df.to_csv("diplomas.csv", index=False, encoding="utf-8")
print("Diploma guardado en diplomas.csv")

df_loaded = pd.read_csv("diplomas.csv", encoding="utf-8")
print("Diplomas cargados:")
print(df_loaded)

Diploma guardado en diplomas.csv
Diplomas cargados:
        id   estudiante                programa                      estado  \
0  DIP-002  Johan Ayala  Ingeniería de Sistemas  Bloqueado - Banderas rojas   

                                                hash  \
0  3687212801ff8c0dd0fc616008f452e1bf1a8a5dc03417...   

                                           historial  
0  [{"estado": "Bloqueado - Banderas rojas", "tim...  


In [ ]:
historial_df = pd.DataFrame(diploma["historial"])
historial_df["timestamp"] = pd.to_datetime(historial_df["timestamp"])

plt.figure(figsize=(8,4))
plt.plot(historial_df["timestamp"], historial_df.index, marker="o")
for i, row in historial_df.iterrows():
    plt.text(row["timestamp"], i, row["estado"], fontsize=9, va="bottom")
plt.title("Ciclo de vida del diploma")
plt.xlabel("Tiempo")
plt.ylabel("Secuencia de estados")
plt.grid(True)
plt.show()